# 合并——笔记部分

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


## 一、append与assign

### 1.append方法

a)利用序列添加行(必须指定name)

In [4]:
df_append=df.loc[:3,['Gender','Height']].copy()
df_append

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167


In [5]:
s = pd.Series({'Gender':'F','Height':188},name='new_row')
df_append.append(s)

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167
new_row,F,188


b)用DataFrame添加表

In [6]:
# 这是自动匹配到对应的大小吗？
df_temp = pd.DataFrame({'Gender':['F','M'],'Height':[188,176]},index=['new_1','new_2'])
df_append.append(df_temp)

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167
new_1,F,188
new_2,M,176


### 2.assign方法

该方法主要用于添加列，列名直接由参数指定：

In [8]:
s = pd.Series(list('abcd'),index=range(4))
df_append.assign(Letter=s)

,Gender,Height,Letter
0,M,173,a
1,F,192,b
2,M,186,c
3,F,167,d


可以一次添加多个列：

In [9]:
#用匿名函数也可以添加
df_append.assign(col1=lambda x:x['Gender']*2,col2=s)

,Gender,Height,col1,col2
0,M,173,MM,a
1,F,192,FF,b
2,M,186,MM,c
3,F,167,FF,d


## 二. combine与update

### 1.combine方法

comine和update都是用于表的填充函数，可以根据某种规则填充

a）填充对象

可以看出combine方法是按照表的顺序轮流进行逐列循环的，而且自动索引对齐，缺失值为NaN，理解这一点很重要

In [13]:
df_combine_1 = df.loc[:1,['Gender','Height']].copy()
df_combine_2 = df.loc[10:11,['Gender','Height']].copy()
df_combine_1.combine(df_combine_2,lambda x,y:print(x,y))

0       M
1       F
10    NaN
11    NaN
Name: Gender, dtype: object 0     NaN
1     NaN
10      M
11      F
Name: Gender, dtype: object
0     173.0
1     192.0
10      NaN
11      NaN
Name: Height, dtype: float64 0       NaN
1       NaN
10    161.0
11    175.0
Name: Height, dtype: float64


,Gender,Height
0,NaN,NaN
1,NaN,NaN
10,NaN,NaN
11,NaN,NaN


In [14]:
df_combine_1 = df.loc[:1,['Gender','Height']].copy()
df_combine_1

,Gender,Height
0,M,173
1,F,192


In [15]:
# 有点不太能理解，里面函数的作用，为何最后全是NaN
# combine是做并集

b)一些例子

In [24]:
# 例子1:根据列均值的大小填充
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'A': [8, 7], 'B': [3, 4]})
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y)

,A,B
0,8,3
1,7,4


In [25]:
#例子2:索引对齐特性（默认状态下，后面的表（df2）没有的行列都会设置为NaN）
df2 = pd.DataFrame({'B':[8,7],'C':[6,5]},index=[1,2])

In [26]:
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y)

,A,B,C
0,NaN,NaN,NaN
1,NaN,8.0,6.0
2,NaN,7.0,5.0


In [29]:
# 这边不明白为什么A不保留1，2,对这个overwrite不是很理解
#overwrite bool, default True
#If True, columns in self that do not exist in other will be overwritten with NaNs.
#如果overwrite为true,在实例中(df1)那些没有出现在另外一个DF(df2)的列会被覆盖成空值

In [28]:
#例子3: 使得df1原来符合条件的值不会被覆盖
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y, 
            overwrite = False)

,A,B,C
0,1.0,NaN,NaN
1,2.0,8.0,6.0
2,NaN,7.0,5.0


In [30]:
#例子4:在新增匹配df2的元素位置填充-1
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,fill_value=-1)

,A,B,C
0,1.0,-1.0,-1.0
1,2.0,8.0,6.0
2,-1.0,7.0,5.0


c)combin_first方法

这个方法作用是用df2填补df1的缺失值，功能比较简单，但很多时候会比combine更常用，下面举两个例子：

In [31]:
df1=pd.DataFrame({'A':[None,0],'B':[None,4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
df1.combine_first(df2)

,A,B
0,1.0,3.0
1,0.0,4.0


In [32]:
df1 = pd.DataFrame({'A': [None, 0], 'B': [4, None]})
df2 = pd.DataFrame({'B': [3, 3], 'C': [1, 1]}, index=[1, 2])
df1.combine_first(df2)

,A,B,C
0,NaN,4.0,NaN
1,0.0,3.0,1.0
2,NaN,3.0,1.0


### 2.update方法

a)三个特点

1）返回的框索引只会与被调用框的一致（默认使用左连接，下一节会介绍）

2）第二个框中的nan元素不会起作用

3）没有返回值，直接在df上操作

b）例子

In [35]:
#例子1: 索引完全对齐情况下的操作
df1 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [400, 500, 600]})
df2 = pd.DataFrame({'B': [4, 5, 6],
                    'C': [7, 8, 9]})
df3=df1.update(df2)
df1

,A,B
0,1,4
1,2,5
2,3,6


In [37]:
df3#没有返回值

In [38]:
# 例子2:部分填充
df1 = pd.DataFrame({'A': ['a', 'b', 'c'],
                    'B': ['x', 'y', 'z']})
df2 = pd.DataFrame({'B': ['d', 'e']}, index=[1,2])
df1.update(df2)
df1

,A,B
0,a,x
1,b,d
2,c,e


In [39]:
# 例子3: 缺失值不会填充
df1 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [400, 500, 600]})
df2 = pd.DataFrame({'B': [4, np.nan, 6]})
df1.update(df2)
df1

,A,B
0,1,4.0
1,2,500.0
2,3,6.0


## concat方法

concat方法可以在两个维度上拼接，默认纵向凭借（axis=0），拼接方式默认外连接

所谓外连接，就是取拼接方向的并集，而'inner'时取拼接方向（若使用默认的纵向拼接，则为列的交集）的交集

下面举一些例子说明其参数：

In [40]:
df1 = pd.DataFrame({'A': ['A0', 'A1'],
                    'B': ['B0', 'B1']},
                    index = [0,1])
df2 = pd.DataFrame({'A': ['A2', 'A3'],
                    'B': ['B2', 'B3']},
                    index = [2,3])
df3 = pd.DataFrame({'A': ['A1', 'A3'],
                    'D': ['D1', 'D3'],
                    'E': ['E1', 'E3']},
                    index = [1,3])

In [42]:
df1

,A,B
0,A0,B0
1,A1,B1


In [43]:
df2

,A,B
2,A2,B2
3,A3,B3


In [44]:
df3

,A,D,E
1,A1,D1,E1
3,A3,D3,E3


In [41]:
# 默认状态拼接
pd.concat([df1,df2])

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [45]:
# axis=1时沿列方向拼接：
pd.concat([df1,df2],axis=1)

,A,B,A,B
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,NaN,NaN,A2,B2
3,NaN,NaN,A3,B3


In [46]:
# join设置为内连接（由于axis=0，因此列取交集）：
pd.concat([df3,df1],join='inner')

,A
1,A1
3,A3
0,A0
1,A1


In [49]:
pd.concat([df3,df1],axis=1,join='inner')

,A,D,E,A,B
1,A1,D1,E1,A1,B1


In [47]:
# join设置为外连接 并？
pd.concat([df3,df1],join='outer',sort=True) #sort设置列排序，默认为False

,A,B,D,E
1,A1,NaN,D1,E1
3,A3,NaN,D3,E3
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN


In [48]:
pd.concat([df3,df1],join='outer')

,A,D,E,B
1,A1,D1,E1,NaN
3,A3,D3,E3,NaN
0,A0,NaN,NaN,B0
1,A1,NaN,NaN,B1


verify_integrity检查列是否唯一

In [50]:
#pd.concat([df3,df1],verify_integrity=True,sort=True) 报错

同样，可以添加Series：

In [52]:
s = pd.Series(['X0', 'X1'], name='X')
pd.concat([df1,s],axis=0)

,A,B,0
0,A0,B0,NaN
1,A1,B1,NaN
0,NaN,NaN,X0
1,NaN,NaN,X1


In [53]:
s = pd.Series(['X0', 'X1'], name='X')
pd.concat([df1,s],axis=1)

,A,B,X
0,A0,B0,X0
1,A1,B1,X1


key参数用于对不同的数据框增加一个标号，便于索引：

In [54]:
pd.concat([df1,df2], keys=['x', 'y'])
pd.concat([df1,df2], keys=['x', 'y']).index

MultiIndex([('x', 0),
            ('x', 1),
            ('y', 2),
            ('y', 3)],
           )

In [55]:
pd.concat([df1,df2], keys=['x', 'y'])

A   B
x 0  A0  B0
  1  A1  B1
y 2  A2  B2
  3  A3  B3